In [1]:
import sys

root_dir = "/home/nisyad/projects/industrial-defect-detection"
sys.path.append(root_dir)

In [2]:
import json
import os
import requests
import torch
from dotenv import load_dotenv
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor
from PIL import Image

from azure.identity import get_bearer_token_provider, DefaultAzureCredential
from openai import AzureOpenAI
from gpt_client import GPTClient, ContentType

from utils import (
    TorchDataset,
    convert_tensor_to_base64,
    convert_to_base64,
    load_local_vision_dataset,
)

import tenacity

load_dotenv()


True

In [3]:
prompt = """You are an expert visual inspector for a manufacturing company that makes glass bottles. You will be shown a top-view image of a glass bottle and your task is to identify if it is defective or not. Think step-by-step - first identify if there is a defect or not. Second, if there is a defect, identify the type of defect. **IF** present, the defect can only be of the following types: 1.broken 2.contamination. Third, explain your reasoning for the defect if present. Finally, identify where the defect is located in the image and provide the relative coordinates (between 0-1) of the bounding box enclosing the defect in the format [x_top, y_top, x_bottom, y_bottom]. Please return your response **strictly** as a valid JSON object with the following format:
{"is_defective": "<yes or no>",
"reason": "<describe the defect. leave empty if is_defective is no>",
"defect_type": "<type of defect. leave empty if is_defective is no>",
"bounding_box": "[x_top, y_top, x_bottom, y_bottom]. leave empty if is_defective is no"}

Note: If  there is no defect or if you are unsure, please return "is_defective": "no" and leave the other fields empty.
"""

prompt

# prompt = "Describe the image in JSON format"

'You are an expert visual inspector for a manufacturing company that makes glass bottles. You will be shown a top-view image of a glass bottle and your task is to identify if it is defective or not. Think step-by-step - first identify if there is a defect or not. Second, if there is a defect, identify the type of defect. **IF** present, the defect can only be of the following types: 1.broken 2.contamination. Third, explain your reasoning for the defect if present. Finally, identify where the defect is located in the image and provide the relative coordinates (between 0-1) of the bounding box enclosing the defect in the format [x_top, y_top, x_bottom, y_bottom]. Please return your response **strictly** as a valid JSON object with the following format:\n{"is_defective": "<yes or no>",\n"reason": "<describe the defect. leave empty if is_defective is no>",\n"defect_type": "<type of defect. leave empty if is_defective is no>",\n"bounding_box": "[x_top, y_top, x_bottom, y_bottom]. leave empt

In [4]:
# def get_messages_body(prompt, base64_image, mime_type):
#     return [
#         {
#             "role": "system",
#             "content": "You are a helpful assistant."
#         },
#         {
#             "role": "user",
#             "content": [
#                     {
#                         "type": "text",
#                         "text": f"{prompt}"
#                     },
#                 {
#                         "type": "image_url",
#                         "image_url": {
#                                 "url": f"data:{mime_type};base64," + f"{base64_image}",
#                         }
#                 }
#             ]
#         }
#     ]

In [5]:
sample_img_path = "datasets/raw/mvtec-ad/bottle/test/good/001.png"

# base64_image = convert_tensor_to_base64(sample_img)
base64_image, mime_type = convert_to_base64(sample_img_path)
print(mime_type)



image/png


In [9]:
gpt_client = GPTClient(
    api_base="https://customvision-dev-aoai.openai.azure.com/",
    deployment_name="gpt4o-003"
)

content = [
    {"type": ContentType.TEXT, "text": prompt},
    {"type": ContentType.IMAGE, "url": f"data:{mime_type};base64," + base64_image}
]

gpt_client.update_messages(content=content)
raw_response = gpt_client.get_response()
outout = gpt_client.extract_output(raw_response)

ChatCompletion(id='chatcmpl-9rYAcb48djwDA3FHYA7IzmHvskU1w', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "is_defective": "yes",\n  "reason": "There is a contamination visible on the inner edge of the bottle opening.",\n  "defect_type": "contamination",\n  "bounding_box": "[0.1, 0.7, 0.2, 0.8]"\n}\n```', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1722549762, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_abc28019ad', usage=CompletionUsage(completion_tokens=69, prompt_tokens=921, total_tokens=990), prompt_filter_results=[{'prompt_index': 0, 'content_filter_result': {'jailbreak': {'filtered': False, 'detected': False}, 'cu

In [8]:
response.choices[0].message.content

'```json\n{\n  "is_defective": "yes",\n  "reason": "There is a contamination visible on the inner edge of the bottle opening.",\n  "defect_type": "contamination",\n  "bounding_box": "[0.1, 0.7, 0.2, 0.8]"\n}\n```'